In [5]:
# Load Library
from pandas import DataFrame
import pandas as pd
import numpy as np
from Bio import Entrez 
import pycurl
import matplotlib.pyplot as plt
import os, sys
from tqdm import tqdm_notebook

In [8]:
df = pd.read_csv("dataset/ncov2019_ncbi08052020.csv")
df.head(1)

,#Organism Name,Organism Groups,BioSample,BioProject,Assembly,Level,Size(Mb),GC%,Replicons,Host,CDS,Neighbors,Release Date,GenBank FTP,RefSeq FTP,Replicons.1
0,Severe acute respiratory syndrome coronavirus 2,Viruses;Other;Coronaviridae,NaN,NaN,GCA_009858895.3,Complete,0.029903,38.0,NC_045512.2/MN908947.3,"human,vertebrates",12,2284.0,2020-01-12T00:00:00Z,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/009...,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009...,NC_045512.2/MN908947.3


In [10]:
#Email for NCBI
Entrez.email = "matin_nuhamunada@ugm.ac.id"
handle = Entrez.efetch(db="nucleotide", id="NC_045512.2")
record = Entrez.read(handle)


NotXMLError: Failed to parse the XML data (syntax error: line 1, column 0). Please make sure that the input data are in XML format.